In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('vgsales.csv')

df.head()

df.shape

df.columns

df.info()

df.describe()

df.isnull().sum()

df['Publisher'] = df['Publisher'].fillna('Unknown')

df = df.dropna(subset=['Year'])

df['Year'] = df['Year'].astype('int64')

text_cols = ['Name','Platform','Genre','Publisher']
df[text_cols] = df[text_cols].astype('string')

df['Name'].dtype

sales_cols = ['NA_Sales','EU_Sales','JP_Sales','Other_Sales']
negative_sales = df[(df[sales_cols]<0).any(axis=1)]

negative_sales.shape[0]

df['sales_sum'] = df[sales_cols].sum(axis=1)

df['Sales_Sum_Check'] = (df['NA_Sales'] + df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales'])

df['Sales_Match'] = np.isclose(df['Sales_Sum_Check'],df['Global_Sales'],atol=0.1)

print("\nGlobal sales mismatch count:", (~df['Sales_Match']).sum())


df['Sales_diff'] = df['Global_Sales'] - df['Sales_Sum_Check']

df['Sales_diff'].describe()

df['Sales_diff'].abs().max()

df.loc[df['Sales_diff'].abs() > 5,
       ['Name', 'Platform', 'Year', 'NA_Sales', 'EU_Sales', 'JP_Sales',
        'Other_Sales', 'Global_Sales', 'Sales_diff']]


invalid_years = df[(df['Year'].notna()) & ((df['Year']<1980) | (df['Year']>2025))]

invalid_years.shape[0]

duplicates = df.duplicated(subset = ['Name','Platform','Year'],keep='first')

print("\nPotential duplicate rows:", duplicates.sum())

dup_rows = df[df.duplicated(subset = ['Name','Platform','Year'],keep='first')]
dup_rows

df = df.drop_duplicates(subset=['Name','Platform','Year'],keep='first')

dup_rows = df[df.duplicated(subset = ['Name','Platform','Year'],keep='first')]
dup_rows

df = df.drop(columns=['sales_sum','Sales_Sum_Check','Sales_Match','Sales_diff'])

df.columns

df.dtypes

df['Platform'].value_counts()

df['Genre'].value_counts()

df.groupby('Platform')['Global_Sales'].sum().sort_values(ascending=False).head(20)

yearly_sales = (df.dropna(subset=['Year']).groupby('Year')['Global_Sales'].sum().sort_values(ascending=False))

plt.figure(figsize=(20,6))
sns.barplot(x=yearly_sales.sort_values(ascending=False).index,y=yearly_sales.sort_values(ascending=False).values)
plt.title('Yearly Sales')
plt.xlabel('Year')
plt.ylabel('Total Sales')
plt.show()

game_sales = df.groupby('Genre')['Global_Sales'].sum().sort_values(ascending=False)

plt.figure(figsize=(20,6))
sns.barplot(x=game_sales.index,y=game_sales.values)
plt.title('Game Sales')
plt.xlabel('Genre')
plt.ylabel('Total Sales')
plt.show()

genre_sales = df.groupby('Genre')['Global_Sales'].sum().sort_values(ascending=False)
genre_sales

plt.figure(figsize=(20,6))
sns.barplot(x=genre_sales.index,y=genre_sales.values)
plt.title('Genre Sales')
plt.xlabel('Genre')
plt.ylabel('Total Sales')
plt.show()

publisher_sales = df.groupby('Publisher')['Global_Sales'].sum().sort_values(ascending=True).head(15)
publisher_sales

plt.figure(figsize=(25,6))
sns.barplot(x=publisher_sales.index,y=publisher_sales.values)
plt.title('Publisher Sales')
plt.xlabel('Publishe')
plt.ylabel('Total Sales')
plt.show()

max_sales = df['NA_Sales'].max(),df['EU_Sales'].max(),df['JP_Sales'].max(),df['Other_Sales'].max()
max_sales

df['Global_Sales'].describe()

df_year = df.dropna(subset=['Year'])

genre_sales_year = df_year.groupby(['Genre','Year'])['Global_Sales'].sum().reset_index()
genre_sales_year

top_genres = (
    df.groupby('Genre')['Global_Sales']
      .sum()
      .sort_values(ascending=False)
      .head(5)
      .index
)

genre_sales_year_top = genre_sales_year[
    genre_sales_year['Genre'].isin(top_genres)
]


trend_table = (
    genre_sales_year_top
    .pivot(index='Year', columns='Genre', values='Global_Sales')
)

trend_table.tail(10)


trend_table.plot(figsize=(12, 6))
plt.title("Top Genres – Global Sales Trend Over Time")
plt.xlabel("Year")
plt.ylabel("Global Sales")
plt.show()

df.to_csv('vg_sales_clean.csv',index=False)

